##MT5 Using Simple Transformers

In [1]:
!pip install folium==0.2.1
!pip install datasets
!pip install transformers
!pip install simpletransformers
!git clone https://gitlab.com/bigirqu/quranqa.git
!pip install -r quranqa/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'quranqa' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import json
import csv
import sklearn
from sklearn.model_selection import train_test_split
import logging
from simpletransformers.t5 import T5Model, T5Args
from datasets import load_dataset, load_metric

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###1) Loading the data, Preparing and Saving it for easier access later.

This Section is needed only once.

In [10]:
datasets =load_dataset('json',
                       data_files={'train': '/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Original/Copy of train_and_augmented.jsonl',
                                   'validation': '/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Original/qrcd_v1.1_dev.jsonl',
                                   'test': '/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Original/Copy of qrcd_v1.1_test_noAnswers.jsonl'})

Using custom data configuration default-8776b62088407bce


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-8776b62088407bce/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
type(datasets)

datasets.dataset_dict.DatasetDict

In [12]:
datasets

DatasetDict({
    train: Dataset({
        features: ['pq_id', 'passage', 'surah', 'verses', 'question', 'answers'],
        num_rows: 1420
    })
    validation: Dataset({
        features: ['pq_id', 'passage', 'surah', 'verses', 'question', 'answers'],
        num_rows: 109
    })
    test: Dataset({
        features: ['pq_id', 'passage', 'surah', 'verses', 'question', 'answers'],
        num_rows: 274
    })
})

In [13]:
type(datasets["train"])

datasets.arrow_dataset.Dataset

In [14]:
datasets["train"]

Dataset({
    features: ['pq_id', 'passage', 'surah', 'verses', 'question', 'answers'],
    num_rows: 1420
})

In [15]:
df_train = pd.DataFrame(datasets["train"])
df_val = pd.DataFrame(datasets["validation"])
df_test = pd.DataFrame(datasets["test"])
df_train.head()

,pq_id,passage,surah,verses,question,answers
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,2,8-16,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'أولئك الذين اشتروا الضلالة بالهدى',..."
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,2,174-176,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,[{'text': 'أولئك الذين اشتروا الضلالة بالهدى و...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,14,47-52,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'ليجزي الله كل نفس ما كسبت', 'start_..."
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,17,12-17,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'كل إنسان ألزمناه طائره في عنقه', 's..."
4,18:29-31_364,وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...,18,29-31,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'من شاء فليؤمن ومن شاء فليكفر', 'sta..."


In [16]:
df_val.head()

,pq_id,passage,surah,verses,question,answers
0,9:60-61_316,إنما الصدقات للفقراء والمساكين والعاملين عليها...,9,60-61,ما هي مصارف الزكاة؟,[{'text': 'للفقراء والمساكين والعاملين عليها و...
1,2:178-179_400,يا أيها الذين آمنوا كتب عليكم القصاص في القتلى...,2,178-179,متى يحل الإسلام دم الشخص؟,[{'text': 'كتب عليكم القصاص في القتلى الحر بال...
2,2:190-194_400,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,2,190-194,متى يحل الإسلام دم الشخص؟,[{'text': 'قاتلوا في سبيل الله الذين يقاتلونكم...
3,2:243-245_400,ألم تر إلى الذين خرجوا من ديارهم وهم ألوف حذر ...,2,243-245,متى يحل الإسلام دم الشخص؟,"[{'text': 'قاتلوا في سبيل الله', 'start_char':..."
4,4:74-76_400,فليقاتل في سبيل الله الذين يشرون الحياة الدنيا...,4,74-76,متى يحل الإسلام دم الشخص؟,"[{'text': 'الذين آمنوا يقاتلون في سبيل الله', ..."


In [17]:
df_test.head() #has no answers

,pq_id,passage,surah,verses,question,answers
0,31:12-19_135,ولقد آتينا لقمان الحكمة أن اشكر لله ومن يشكر ف...,31,12-19,ما هي وصايا لقمان لابنه؟,[]
1,2:234-237_124,والذين يتوفون منكم ويذرون أزواجا يتربصن بأنفسه...,2,234-237,كم مدة عدة الأرملة؟,[]
2,2:87-88_241,ولقد آتينا موسى الكتاب وقفينا من بعده بالرسل و...,2,87-88,من هم الملائكة المذكورون في القرآن؟,[]
3,2:97-101_241,قل من كان عدوا لجبريل فإنه نزله على قلبك بإذن ...,2,97-101,من هم الملائكة المذكورون في القرآن؟,[]
4,2:102-103_241,واتبعوا ما تتلو الشياطين على ملك سليمان وما كف...,2,102-103,من هم الملائكة المذكورون في القرآن؟,[]


In [18]:
df_train.question[45]

'ما هي انواع الحيوانات التي ذكرت في القرآن؟'

In [19]:
df_train.answers[45] #list of dictionaries

[{'text': 'العجل', 'start_char': 98}, {'text': 'السلوى', 'start_char': 348}]

In [20]:
def prepare_dataset(dataset):
  """ returns a prepared copy of the dataset given after preparing it and getting the columns we want for the mt5 model"""
  dataset_pepared = dataset.copy()
  dataset_pepared = dataset_pepared.explode('answers').reset_index(drop=True) # make a row for each answer dict
  dataset_pepared = dataset_pepared.join(pd.json_normalize(dataset_pepared.answers)).drop(columns=['answers']) #unzip the answer dict into 2 columns and drop the original
  dataset_pepared = dataset_pepared[["pq_id", "passage", "question", "text"]] # for mt5 model we only need  these columns
  return dataset_pepared


In [21]:
train = prepare_dataset(df_train)
train.head()

,pq_id,passage,question,text
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,أولئك الذين اشتروا الضلالة بالهدى
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,ليجزي الله كل نفس ما كسبت
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,كل إنسان ألزمناه طائره في عنقه
4,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها


In [22]:
val = prepare_dataset(df_val)
val.head()

,pq_id,passage,question,text
0,9:60-61_316,إنما الصدقات للفقراء والمساكين والعاملين عليها...,ما هي مصارف الزكاة؟,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...
1,2:178-179_400,يا أيها الذين آمنوا كتب عليكم القصاص في القتلى...,متى يحل الإسلام دم الشخص؟,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...
2,2:190-194_400,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,متى يحل الإسلام دم الشخص؟,قاتلوا في سبيل الله الذين يقاتلونكم
3,2:243-245_400,ألم تر إلى الذين خرجوا من ديارهم وهم ألوف حذر ...,متى يحل الإسلام دم الشخص؟,قاتلوا في سبيل الله
4,4:74-76_400,فليقاتل في سبيل الله الذين يشرون الحياة الدنيا...,متى يحل الإسلام دم الشخص؟,الذين آمنوا يقاتلون في سبيل الله


In [23]:
df_test.shape

(274, 6)

In [24]:
def prepare_test_dataset(dataset):
  """ returns a prepared copy of the dataset given after preparing it and getting the columns we want for the mt5 model"""
  dataset_pepared = dataset.copy()
  dataset_pepared = dataset_pepared[["pq_id", "passage", "question"]] # for test mt5 model we only need  these columns
  return dataset_pepared

In [25]:
test = prepare_test_dataset(df_test)
test.head()

,pq_id,passage,question
0,31:12-19_135,ولقد آتينا لقمان الحكمة أن اشكر لله ومن يشكر ف...,ما هي وصايا لقمان لابنه؟
1,2:234-237_124,والذين يتوفون منكم ويذرون أزواجا يتربصن بأنفسه...,كم مدة عدة الأرملة؟
2,2:87-88_241,ولقد آتينا موسى الكتاب وقفينا من بعده بالرسل و...,من هم الملائكة المذكورون في القرآن؟
3,2:97-101_241,قل من كان عدوا لجبريل فإنه نزله على قلبك بإذن ...,من هم الملائكة المذكورون في القرآن؟
4,2:102-103_241,واتبعوا ما تتلو الشياطين على ملك سليمان وما كف...,من هم الملائكة المذكورون في القرآن؟


Now we save the dataframes to csv files for easier loading and accessing later.

In [26]:
train.to_csv('/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Before Preprocessing/training-data-bp.csv')
val.to_csv('/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Before Preprocessing/validation-data-bp.csv')
test.to_csv('/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Before Preprocessing/test-data-bp.csv')

###Preprocessing using Camel Tools.

In [27]:
%pip install camel-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 114 kB 4.2 MB/s 
     |████████████████████████████████| 418 kB 24.4 MB/s 
     |████████████████████████████████| 175 kB 62.7 MB/s 
  Created wheel for camel-kenlm: filename=camel_kenlm-2021.12.27-cp37-cp37m-linux_x86_64.whl size=2336052 sha256=572162e960bc33ce83afd368d17df037553bbaa24bd51af7af73170f9a1a439b
  Stored in directory: /root/.cache/pip/wheels/db/72/74/982f8c435f15b7feaf6dc8a03e212ff34e93f1f2d747059332
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=7534a8de7f838a403c174b766b9bf69ee69c28e0770c8f537c5e3dd98735fbe8
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built camel-kenlm emoji


In [28]:
import os
#%mkdir /content/drive/MyDrive/MT5/camel_tools #needed only once

In [29]:
os.environ['CAMELTOOLS_DATA'] = '/content/drive/MyDrive/MT5/camel_tools'

#!export | camel_data -i all #needed only once

In [30]:
# -*- coding: utf-8 -*- #to be able to process arabic
import pandas as pd
import re
from camel_tools.utils.dediac import dediac_ar
from camel_tools.utils.charsets import AR_LETTERS_CHARSET
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.disambig.mle import MLEDisambiguator
mle_pretrained = MLEDisambiguator.pretrained()

First, let's load our data and investigate it.

In [31]:
data_origin_train = pd.read_csv("/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Before Preprocessing/training-data-bp.csv", index_col=0)
data_train = data_origin_train.copy()
data_train.head()

,pq_id,passage,question,text
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,أولئك الذين اشتروا الضلالة بالهدى
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,ليجزي الله كل نفس ما كسبت
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,كل إنسان ألزمناه طائره في عنقه
4,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها


In [32]:
data_origin_val = pd.read_csv("/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Before Preprocessing/validation-data-bp.csv", index_col=0)
data_val = data_origin_val.copy()
data_val.head()

,pq_id,passage,question,text
0,9:60-61_316,إنما الصدقات للفقراء والمساكين والعاملين عليها...,ما هي مصارف الزكاة؟,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...
1,2:178-179_400,يا أيها الذين آمنوا كتب عليكم القصاص في القتلى...,متى يحل الإسلام دم الشخص؟,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...
2,2:190-194_400,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,متى يحل الإسلام دم الشخص؟,قاتلوا في سبيل الله الذين يقاتلونكم
3,2:243-245_400,ألم تر إلى الذين خرجوا من ديارهم وهم ألوف حذر ...,متى يحل الإسلام دم الشخص؟,قاتلوا في سبيل الله
4,4:74-76_400,فليقاتل في سبيل الله الذين يشرون الحياة الدنيا...,متى يحل الإسلام دم الشخص؟,الذين آمنوا يقاتلون في سبيل الله


In [33]:
data_origin_test = pd.read_csv("/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Before Preprocessing/test-data-bp.csv", index_col=0)
data_test = data_origin_test.copy()
data_test.head()

,pq_id,passage,question
0,31:12-19_135,ولقد آتينا لقمان الحكمة أن اشكر لله ومن يشكر ف...,ما هي وصايا لقمان لابنه؟
1,2:234-237_124,والذين يتوفون منكم ويذرون أزواجا يتربصن بأنفسه...,كم مدة عدة الأرملة؟
2,2:87-88_241,ولقد آتينا موسى الكتاب وقفينا من بعده بالرسل و...,من هم الملائكة المذكورون في القرآن؟
3,2:97-101_241,قل من كان عدوا لجبريل فإنه نزله على قلبك بإذن ...,من هم الملائكة المذكورون في القرآن؟
4,2:102-103_241,واتبعوا ما تتلو الشياطين على ملك سليمان وما كف...,من هم الملائكة المذكورون في القرآن؟


In [34]:
stopwords = []
with open('/content/drive/MyDrive/MT5/camel_tools/list.txt', encoding='utf8') as file:
    for line in file:
        stopwords.append(line.strip())
print(stopwords)
print(type(stopwords)) 

['،', 'ء', 'ءَ', 'آ', 'آب', 'آذار', 'آض', 'آل', 'آمينَ', 'آناء', 'آنفا', 'آه', 'آهاً', 'آهٍ', 'آهِ', 'أ', 'أبدا', 'أبريل', 'أبو', 'أبٌ', 'أجل', 'أجمع', 'أحد', 'أخبر', 'أخذ', 'أخو', 'أخٌ', 'أربع', 'أربعاء', 'أربعة', 'أربعمئة', 'أربعمائة', 'أرى', 'أسكن', 'أصبح', 'أصلا', 'أضحى', 'أطعم', 'أعطى', 'أعلم', 'أغسطس', 'أفريل', 'أفعل به', 'أفٍّ', 'أقبل', 'أكتوبر', 'أل', 'ألا', 'ألف', 'ألفى', 'أم', 'أما', 'أمام', 'أمامك', 'أمامكَ', 'أمد', 'أمس', 'أمسى', 'أمّا', 'أن', 'أنا', 'أنبأ', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'أنتِ', 'أنشأ', 'أنه', 'أنًّ', 'أنّى', 'أهلا', 'أو', 'أوت', 'أوشك', 'أول', 'أولئك', 'أولاء', 'أولالك', 'أوّهْ', 'أى', 'أي', 'أيا', 'أيار', 'أيضا', 'أيلول', 'أين', 'أيّ', 'أيّان', 'أُفٍّ', 'ؤ', 'إحدى', 'إذ', 'إذا', 'إذاً', 'إذما', 'إذن', 'إزاء', 'إلى', 'إلي', 'إليكم', 'إليكما', 'إليكنّ', 'إليكَ', 'إلَيْكَ', 'إلّا', 'إمّا', 'إن', 'إنَّ', 'إى', 'إياك', 'إياكم', 'إياكما', 'إياكن', 'إيانا', 'إياه', 'إياها', 'إياهم', 'إياهما', 'إياهن', 'إياي', 'إيهٍ', 'ئ', 'ا', 'ا?', 'ا?ى', 'االا', 'االتى', 'اب

In [35]:
data_train.iloc[0][1]

'ومن الناس من يقول آمنا بالله وباليوم الآخر وما هم بمؤمنين. يخادعون الله والذين آمنوا وما يخدعون إلا أنفسهم وما يشعرون. في قلوبهم مرض فزادهم الله مرضا ولهم عذاب أليم بما كانوا يكذبون. وإذا قيل لهم لا تفسدوا في الأرض قالوا إنما نحن مصلحون. ألا إنهم هم المفسدون ولكن لا يشعرون. وإذا قيل لهم آمنوا كما آمن الناس قالوا أنؤمن كما آمن السفهاء ألا إنهم هم السفهاء ولكن لا يعلمون. وإذا لقوا الذين آمنوا قالوا آمنا وإذا خلوا إلى شياطينهم قالوا إنا معكم إنما نحن مستهزئون. الله يستهزئ بهم ويمدهم في طغيانهم يعمهون. أولئك الذين اشتروا الضلالة بالهدى فما ربحت تجارتهم وما كانوا مهتدين.'

In [36]:
data_train.shape

(1722, 4)

In [37]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1722 entries, 0 to 1721
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pq_id     1722 non-null   object
 1   passage   1722 non-null   object
 2   question  1722 non-null   object
 3   text      1722 non-null   object
dtypes: object(4)
memory usage: 67.3+ KB


#### 1) Cleaning our data: 
- Remove URLS
- Mentions
- Emotions
- Symbols
- Diacritical marks
- Shapes
- Unwanted Punctuation (basicly anything that is not an arabic word or a number).

In [38]:
def clean_text(text):
    sentence_ar_dediac = dediac_ar(text) #dediacritization using camel tools
    sentence_ar_dediac = re.sub(r"\S*https?:\S*|@\S+", "", sentence_ar_dediac) #remove URLs and mentions even if they are in paranthesis or brackets
    no_punc = ""
    for char in sentence_ar_dediac:
        if char in (list(AR_LETTERS_CHARSET)+[" "]): #removing any thing that is not an arabic letter
            no_punc = no_punc + char
    return no_punc

After Cleaning,
#### 2) Normalizing our data
- Orthographic Normalization
- Unicode Normalization

In [39]:
def normalize_text(text):
    cleaned_text = normalize_alef_maksura_ar(text)
    cleaned_text = normalize_alef_ar(cleaned_text)
    cleaned_text = normalize_teh_marbuta_ar(cleaned_text)
    cleaned_text = normalize_unicode(cleaned_text)
    
    return cleaned_text

#### 3) Data Enrichement
- Tokenization
- Removing stop words
- Morphology Analysis
- Lemmatization

In [40]:
def enrichement(text, stopwords=stopwords, mle_pretrained=mle_pretrained):
    tokens = simple_word_tokenize(text) #tokenize the sentence
    tokenized = []
    for token in tokens: #remove stop words
        if token not in stopwords:
            tokenized.append(token)     
    disambig = mle_pretrained.disambiguate(tokenized) #return each word to its base form
    lemmas = [d.analyses[0].analysis['lex'] for d in disambig]
    whole_sent = " ".join(lemmas) # join to get the whole sentence
    whole_sent_dediac = dediac_ar(whole_sent) #diacritize the sentence 
    return whole_sent_dediac

In [41]:
def preprocess_text(text, clean_text=clean_text, normalize_text=normalize_text, enrichement=enrichement):
    '''
    Inputs:
    text: string in which it will preprocess.
    
    Outputs:
    cleaned_text: the string after it was processed.
    '''
    
    #let's first start with cleaning 
    cleaned = clean_text(text)
            
    # now, for the normalization part
    normalized = normalize_text(cleaned)
    
    #enrichement
    enriched = enrichement(normalized)
    
    return enriched

In [42]:
def preprocess_data(data, columns, preprocess_text=preprocess_text):
    '''
    Inputs:
    data: is the dataframe we want to apply our function on
    columns: cols in data that we want to preprocess
    
    Outputs:
    data: the dataframe after applying the cleaning and normalization on each cell.
    
    '''
    for row in range(data.shape[0]):
        for column in columns:
            for_cleaning = str(data.iloc[row][column])
            data.iloc[row][column] = preprocess_text(for_cleaning)
    return data

In [43]:
train = preprocess_data(data_train, ['question'])
train.head()

,pq_id,passage,question,text
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,أولئك الذين اشتروا الضلالة بالهدى
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,ليجزي الله كل نفس ما كسبت
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,كل إنسان ألزمناه طائره في عنقه
4,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها


In [44]:
val = preprocess_data(data_val, ['question'])
val.head()

,pq_id,passage,question,text
0,9:60-61_316,إنما الصدقات للفقراء والمساكين والعاملين عليها...,مصرف زكاة,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...
1,2:178-179_400,يا أيها الذين آمنوا كتب عليكم القصاص في القتلى...,متى أحل إسلام دم شخص,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...
2,2:190-194_400,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,متى أحل إسلام دم شخص,قاتلوا في سبيل الله الذين يقاتلونكم
3,2:243-245_400,ألم تر إلى الذين خرجوا من ديارهم وهم ألوف حذر ...,متى أحل إسلام دم شخص,قاتلوا في سبيل الله
4,4:74-76_400,فليقاتل في سبيل الله الذين يشرون الحياة الدنيا...,متى أحل إسلام دم شخص,الذين آمنوا يقاتلون في سبيل الله


In [45]:
test = preprocess_data(data_test, ['question'])
test.head()

,pq_id,passage,question
0,31:12-19_135,ولقد آتينا لقمان الحكمة أن اشكر لله ومن يشكر ف...,وصية لقمان ٱبن
1,2:234-237_124,والذين يتوفون منكم ويذرون أزواجا يتربصن بأنفسه...,مدة عدة أرملة
2,2:87-88_241,ولقد آتينا موسى الكتاب وقفينا من بعده بالرسل و...,ملأك مذكور قرآن
3,2:97-101_241,قل من كان عدوا لجبريل فإنه نزله على قلبك بإذن ...,ملأك مذكور قرآن
4,2:102-103_241,واتبعوا ما تتلو الشياطين على ملك سليمان وما كف...,ملأك مذكور قرآن


In [46]:
train.to_csv('/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Preprocessed using Camel/data-preprocessed-q.csv') #preprocessed all vs preprocessed q
val.to_csv('/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Preprocessed using Camel/data-val-preprocessed-q.csv')
test.to_csv('/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Preprocessed using Camel/data-test-preprocessed-q.csv')

### Prepare the Data for the mt5 model:
- Simple transformers implementation of mT5 model expects a data to be a dataframe with 3 columns: \<prefix>, \<input_text>, \<target_text>
- \<prefix>: A string indicating the task to perform. (E.g. “question”, “stsb”, “summarization”)

- \<input_text>: The input text sequence (we will use question and context as input_text )

- \<target_text>: The target sequence (we will use text as output_text )

In [47]:
train = pd.read_csv("/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Preprocessed using Camel/data-preprocessed-q.csv", index_col=0) #preprocessed all vs preprocessed q
val = pd.read_csv("/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Preprocessed using Camel/data-val-preprocessed-q.csv", index_col=0)
test = pd.read_csv("/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Preprocessed using Camel/data-test-preprocessed-q.csv", index_col=0)
train.head()

,pq_id,passage,question,text
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,أولئك الذين اشتروا الضلالة بالهدى
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,ليجزي الله كل نفس ما كسبت
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,كل إنسان ألزمناه طائره في عنقه
4,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا حاسب عذب ضال قيامة ضلل الله ما هادي قول ...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها


In [48]:
def prepare_dataset_mt5(dataset):
  """ returns a dataframe of the data in the required shape for the mt5 model which is 3 columns <prefix>, <input_text>, <target_text>"""
  dataset_mt5 = pd.DataFrame()
  dataset_mt5['input_text'] = dataset.apply(lambda row: "question: %s  context: %s " % (row['question'], row['passage']), axis=1)
  dataset_mt5['target_text'] = dataset.apply(lambda row: "%s " % (row['text']), axis=1)
  dataset_mt5['prefix'] = "answer question"
  return dataset_mt5

In [49]:
train_mt5 = prepare_dataset_mt5(train)
train_mt5.head()

,input_text,target_text,prefix
0,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,أولئك الذين اشتروا الضلالة بالهدى,answer question
1,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...,answer question
2,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,ليجزي الله كل نفس ما كسبت,answer question
3,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,كل إنسان ألزمناه طائره في عنقه,answer question
4,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عل...,answer question


In [50]:
val_mt5 = prepare_dataset_mt5(val)
val_mt5.head()

,input_text,target_text,prefix
0,question: مصرف زكاة context: إنما الصدقات للف...,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...,answer question
1,question: متى أحل إسلام دم شخص context: يا أي...,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...,answer question
2,question: متى أحل إسلام دم شخص context: وقاتل...,قاتلوا في سبيل الله الذين يقاتلونكم,answer question
3,question: متى أحل إسلام دم شخص context: ألم ت...,قاتلوا في سبيل الله,answer question
4,question: متى أحل إسلام دم شخص context: فليقا...,الذين آمنوا يقاتلون في سبيل الله,answer question


In [60]:
def prepare_dataset_mt5_test(dataset):
  """ returns a dataframe of the data in the required shape for the mt5 model which is 3 columns <prefix>, <input_text>, <target_text>"""
  dataset_mt5 = pd.DataFrame()
  dataset_mt5['input_text'] = dataset.apply(lambda row: "question: %s  context: %s " % (row['question'], row['passage']), axis=1)
  dataset_mt5['prefix'] = "answer question"
  return dataset_mt5

In [61]:
test_mt5 = prepare_dataset_mt5_test(test)
test_mt5.head()

,input_text,prefix
0,question: وصية لقمان ٱبن context: ولقد آتينا ...,answer question
1,question: مدة عدة أرملة context: والذين يتوفو...,answer question
2,question: ملأك مذكور قرآن context: ولقد آتينا...,answer question
3,question: ملأك مذكور قرآن context: قل من كان ...,answer question
4,question: ملأك مذكور قرآن context: واتبعوا ما...,answer question


In [53]:
train_mt5.astype(str)


,input_text,target_text,prefix
0,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,أولئك الذين اشتروا الضلالة بالهدى,answer question
1,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...,answer question
2,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,ليجزي الله كل نفس ما كسبت,answer question
3,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,كل إنسان ألزمناه طائره في عنقه,answer question
4,question: لماذا حاسب عذب ضال قيامة ضلل الله ما...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عل...,answer question
...,...,...,...
1717,question: سيد محمد تكلم لجن context: قل أوحي ...,قل أوحي إلي أنه استمع نفر من الجن,answer question
1718,question: فأر ظهار إسلام context: قد سمع الله...,تحرير رقبة من قبل أن يتماسا ذلكم توعظون به وال...,answer question
1719,question: صفة المطففين context: ويل للمطففين....,الذين إذا اكتالوا على الناس يستوفون. وإذا كالو...,answer question
1720,question: جرى ليل قدر context: إنا أنزلناه في...,ليلة القدر خير من ألف شهر. تنزل الملائكة والرو...,answer question


In [54]:
val_mt5.astype(str)

,input_text,target_text,prefix
0,question: مصرف زكاة context: إنما الصدقات للف...,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...,answer question
1,question: متى أحل إسلام دم شخص context: يا أي...,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...,answer question
2,question: متى أحل إسلام دم شخص context: وقاتل...,قاتلوا في سبيل الله الذين يقاتلونكم,answer question
3,question: متى أحل إسلام دم شخص context: ألم ت...,قاتلوا في سبيل الله,answer question
4,question: متى أحل إسلام دم شخص context: فليقا...,الذين آمنوا يقاتلون في سبيل الله,answer question
...,...,...,...
123,question: سمح إسلام حر ٱعتقاد دخول إسلام cont...,من اهتدى فلنفسه ومن ضل فإنما يضل عليها,answer question
124,question: سمح إسلام حر ٱعتقاد دخول إسلام cont...,قل يا أيها الكافرون. لا أعبد ما تعبدون. ولا أن...,answer question
125,question: أثم حاكم حكم ما أنزل الله شرع conte...,من لم يحكم بما أنزل الله فأولئك هم الكافرون,answer question
126,question: أثم حاكم حكم ما أنزل الله شرع conte...,من لم يحكم بما أنزل الله فأولئك هم الظالمون,answer question


In [62]:
test_mt5.astype(str)

,input_text,prefix
0,question: وصية لقمان ٱبن context: ولقد آتينا ...,answer question
1,question: مدة عدة أرملة context: والذين يتوفو...,answer question
2,question: ملأك مذكور قرآن context: ولقد آتينا...,answer question
3,question: ملأك مذكور قرآن context: قل من كان ...,answer question
4,question: ملأك مذكور قرآن context: واتبعوا ما...,answer question
...,...,...
269,question: أشار قرآن عضو عقل إنسان context: وإ...,answer question
270,question: أشار قرآن عضو عقل إنسان context: وي...,answer question
271,question: لماذا تعادل شهادة مر واحد شهادة رجل ...,answer question
272,question: لماذا عبد أتى الله علم سور كهف قتل غ...,answer question


In [63]:
print(train_mt5.shape)
print(val_mt5.shape)
print(test_mt5.shape)

(1722, 3)
(128, 3)
(274, 2)


In [57]:
train_mt5['input_text'][64]

'question: نوع حيوان ذكر قرآن  context: يسألونك ماذا أحل لهم قل أحل لكم الطيبات وما علمتم من الجوارح مكلبين تعلمونهن مما علمكم الله فكلوا مما أمسكن عليكم واذكروا اسم الله عليه واتقوا الله إن الله سريع الحساب. اليوم أحل لكم الطيبات وطعام الذين أوتوا الكتاب حل لكم وطعامكم حل لهم والمحصنات من المؤمنات والمحصنات من الذين أوتوا الكتاب من قبلكم إذا آتيتموهن أجورهن محصنين غير مسافحين ولا متخذي أخدان ومن يكفر بالإيمان فقد حبط عمله وهو في الآخرة من الخاسرين. '

In [58]:
labels = list(val_mt5['target_text'])
labels

['للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل ',
 'كتب عليكم القصاص في القتلى الحر بالحر والعبد بالعبد والأنثى بالأنثى ',
 'قاتلوا في سبيل الله الذين يقاتلونكم ',
 'قاتلوا في سبيل الله ',
 'الذين آمنوا يقاتلون في سبيل الله ',
 'الذين كفروا يقاتلون في سبيل الطاغوت فقاتلوا أولياء الشيطان ',
 'فقاتل في سبيل الله ',
 'حرض المؤمنين عسى الله أن يكف بأس الذين كفروا ',
 'ودوا لو تكفرون كما كفروا فتكونون سواء فلا تتخذوا منهم أولياء حتى يهاجروا في سبيل الله فإن تولوا فخذوهم واقتلوهم حيث وجدتموهم ولا تتخذوا منهم وليا ولا نصيرا ',
 'إن لم يعتزلوكم ويلقوا إليكم السلم ويكفوا أيديهم فخذوهم واقتلوهم حيث ثقفتموهم ',
 'الذين يحاربون الله ورسوله ',
 'يسعون في الأرض فسادا ',
 'قاتلوهم حتى لا تكون فتنة ويكون الدين كله لله ',
 'فإذا انسلخ الأشهر الحرم فاقتلوا المشركين حيث وجدتموهم وخذوهم واحصروهم واقعدوا لهم كل مرصد ',
 'إن نكثوا أيمانهم من بعد عهدهم وطعنوا في دينكم فقاتلوا أئمة الكفر ',
 'قاتلوا الذين لا يؤمنون بالله ولا باليوم الآخر ولا يحرمون ما حرم ال

### Train the Model

In [13]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [14]:
from simpletransformers.t5 import T5Model, T5Args

model_args = T5Args() 
model_args.learning_rate = 0.0001
model_args.max_seq_length = 128                             #Maximum sequence length the model will support.
model_args.train_batch_size = 8                             #prefered to be a power of 2
model_args.eval_batch_size = 8
model_args.num_train_epochs = 40                            #The number of epochs the model will be trained for.
model_args.evaluate_generated_text = True                   #Generate sequences for evaluation.
model_args.evaluate_during_training = True                  #Set to True to perform evaluation while training models. Make sure eval data is passed to the training method if enabled.
model_args.evaluate_during_training_verbose = True          #Print results from evaluation during training.
model_args.use_multiprocessing = False                      #If True, multiprocessing will be used when converting data into features. Enabling can speed up processing, but may be unstable in certain cases.
model_args.fp16 = False                                     #	Whether or not fp16 mode should be used. Requires NVidia Apex library.
model_args.save_steps = -1                                  #Save a model checkpoint at every specified number of steps. Set to -1 to disable. default 2000
model_args.save_eval_checkpoints = False                    #Save a model checkpoint for every evaluation performed.
model_args.save_model_every_epoch = False                   #	Save a model checkpoint at the end of every epoch.
model_args.no_cache = True                                  #Cache features to disk.
model_args.reprocess_input_data = True                      #If True, the input data will be reprocessed even if a cached file of the input data exists in the cache_dir.
model_args.preprocess_inputs = True                         #Automatically add : and < /s> tokens to train_model() and eval_model() inputs. Automatically add < /s> to each string in to_predict in predict().
model_args.overwrite_output_dir = True                      #If True, the trained model will be saved to the ouput_dir and will overwrite existing saved models in the same directory.
model_args.num_return_sequences = 1                         #The number of samples to generate.
model_args.output_dir = "/content/drive/MyDrive/MT5/outputs/"
model_args.best_model_dir = "/content/drive/MyDrive/MT5/outputs/best_model"
model_args.wandb_project = "MT5 QA"                         #Name of W&B project. This will log all hyperparameter values, training losses, and evaluation metrics to the given project.

model = T5Model("mt5", "google/mt5-base", args=model_args)

In [15]:
!nvidia-smi

Sat Jun 18 23:09:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:386: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [17]:
# Train the model
model.train_model(train_mt5, eval_data=val_mt5)


INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/1722 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3557: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

wandb: Currently logged in as: gp-2022. Use `wandb login --relogin` to force relogin


...

  0%|          | 0/128 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/16 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/128 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 1.7628437289968133}
INFO:simpletransformers.t5.t5_model: Training of google/mt5-base model complete. Saved to /content/drive/MyDrive/MT5/outputs/.


(8640,
 {'global_step': [216,
   432,
   648,
   864,
   1080,
   1296,
   1512,
   1728,
   1944,
   2000,
   2160,
   2376,
   2592,
   2808,
   3024,
   3240,
   3456,
   3672,
   3888,
   4000,
   4104,
   4320,
   4536,
   4752,
   4968,
   5184,
   5400,
   5616,
   5832,
   6000,
   6048,
   6264,
   6480,
   6696,
   6912,
   7128,
   7344,
   7560,
   7776,
   7992,
   8000,
   8208,
   8424,
   8640],
  'eval_loss': [5.16186797618866,
   2.5748846605420113,
   2.103583723306656,
   2.0105499401688576,
   1.8073920421302319,
   1.7922522202134132,
   1.6096444744616747,
   1.504730386659503,
   1.4500889144837856,
   1.4606640916317701,
   1.4725703299045563,
   1.383589493110776,
   1.355655612424016,
   1.3504034020006657,
   1.3421080950647593,
   1.3206024058163166,
   1.3209061920642853,
   1.3425900880247355,
   1.3256670199334621,
   1.3574415650218725,
   1.3704344555735588,
   1.3432922195643187,
   1.3767169937491417,
   1.402818389236927,
   1.4197168238461018,
   1

In [18]:
# Optional: Evaluate the model. We'll test it properly anyway.
results = model.eval_model(val_mt5, verbose=True)
results

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/128 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/16 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/128 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 1.7628437289968133}


{'eval_loss': 1.7628437289968133}

In [20]:
to_predict_val = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(val_mt5["prefix"].tolist(), val_mt5["input_text"].tolist())
]
to_predict_val[62]
#type(to_predict[0])

'answer question: question: أهل الله قوم  context: وفي موسى إذ أرسلناه إلى فرعون بسلطان مبين. فتولى بركنه وقال ساحر أو مجنون. فأخذناه وجنوده فنبذناهم في اليم وهو مليم. وفي عاد إذ أرسلنا عليهم الريح العقيم. ما تذر من شيء أتت عليه إلا جعلته كالرميم. وفي ثمود إذ قيل لهم تمتعوا حتى حين. فعتوا عن أمر ربهم فأخذتهم الصاعقة وهم ينظرون. فما استطاعوا من قيام وما كانوا منتصرين. وقوم نوح من قبل إنهم كانوا قوما فاسقين. '

In [21]:
y = model.predict(to_predict_val)

Generating outputs:   0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3557: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f197d33a440> on tensor will not be serial

Decoding outputs:   0%|          | 0/128 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f1976cd64d0> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f1976cd6680> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f1976cd6830> on tensor will not be serialized.  If this is exp

In [22]:
type(y)

list

In [23]:
def create_final_csv(y):
  final = pd.DataFrame()
  final["pq_id"] = val["pq_id"]
  final["answer"] = pd.Series(y)
  final["rank"] = 1
  final["score"] = 1
  return final

final = create_final_csv(y)
final.head()

,pq_id,answer,rank,score
0,9:60-61_316,يؤذون النبي ويقولون هو أذن قل أذن خير,1,1
1,2:178-179_400,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...,1,1
2,2:190-194_400,الشهر الحرام بالشهر الحرام والحرمات قصاص فمن ا...,1,1
3,2:243-245_400,من ذا يقرض الله قرضا حسنا فيضاعفه له أضعافا,1,1
4,4:74-76_400,من يقاتل في سبيل الله فيقتل أو يغلب فسوف نؤ,1,1


In [24]:
final.to_csv('/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Predictions/final_predicted.csv')

In [25]:
from itertools import islice
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        targets = ["answer", "rank", "score"]
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            key = rows['pq_id']
            result = {key: rows[key] for key in rows.keys() if key in targets}
            data[key] = [result]
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))

In [26]:
# Driver Code
 
# Decide the two file paths according to your
# computer system
csvFilePath = r'/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Predictions/final_predicted.csv'
jsonFilePath = r'/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Predictions/final_predicted.json'
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath)

In [27]:
!cp /content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/quranqa22_submission_checker.py .
!python quranqa22_submission_checker.py\
    --run_file="/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Predictions/final_predicted.json"

The run file is correct.


In [28]:
!cp /content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/quranqa22_eval.py .
!python quranqa22_eval.py \
--gold_answers_file "/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Original/qrcd_v1.1_dev.jsonl" \
--run_file "/content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Predictions/final_predicted.json"

[2022-06-19 01:11:49,103 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from /content/drive/MyDrive/MT5/MT5-Model-on-Quran-Dataset/Data/Data Original/qrcd_v1.1_dev.jsonl
The run file is correct.
{"pRR": 0.39391797174299437, "exact_match": 0.21100917431192662, "f1": 0.39391797174299437}


In [29]:
#bleu_score_mt5 = sacrebleu.corpus_bleu(y, labels)
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
import numpy as np
import nltk
nltk.download('punkt')

def get_bleu_scores(y, labels):

  length_f = len(y)
  unigram = np.array([])
  bigram = np.array([])
  trigram = np.array([])
  quadgram = np.array([])
  Cumulative_2_gram = np.array([])
  Cumulative_3_gram = np.array([])
  Cumulative_4_gram = np.array([])

  for pred, lab in zip(y, labels):
    x1 = word_tokenize(lab)
    x2 = word_tokenize(pred)
    unigram = np.append(unigram, sentence_bleu([x1], x2, weights=(1,0,0,0)))
    bigram = np.append(bigram, sentence_bleu([x1], x2, weights=(0,1,0,0)))
    trigram = np.append(trigram, sentence_bleu([x1], x2, weights=(0,0,1,0)))
    quadgram = np.append(quadgram, sentence_bleu([x1], x2, weights=(0,0,1,0)))
    Cumulative_2_gram = np.append(Cumulative_2_gram, sentence_bleu([x1], x2, weights=(0.5, 0.5, 0, 0)))
    Cumulative_3_gram = np.append(Cumulative_3_gram, sentence_bleu([x1], x2, weights=(0.33, 0.33,0.33, 0)))
    Cumulative_4_gram = np.append(Cumulative_4_gram, sentence_bleu([x1], x2, weights=(0.25, 0.25, 0.25, 0.25)))

  return (np.sum(unigram)/ length_f), (np.sum(bigram)/ length_f), (np.sum(trigram)/ length_f), (np.sum(quadgram)/ length_f), (np.sum(Cumulative_2_gram)/ length_f), (np.sum(Cumulative_3_gram)/ length_f), (np.sum(Cumulative_4_gram)/ length_f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
bleu_1, bi, tri, quad, bleu_2, bleu_3, bleu_4 = get_bleu_scores(y, labels);
print('Individual N-gram scores')
print('Bleu score-uni: ', bleu_1)
print('Bleu score-bi: ',bi)
print('Bleu score-tri: ',tri)
print('Bleu score-quad: ',quad)
print('---------------------------------------')
print('Cumulative N-gram scores')
print('Bleu-2: ', bleu_2)
print('Bleu-3: ', bleu_3);
print('Bleu-4: ', bleu_4);



Individual N-gram scores
Bleu score-uni:  0.3199468262479621
Bleu score-bi:  0.2822666506853665
Bleu score-tri:  0.26918573601840873
Bleu score-quad:  0.26918573601840873
---------------------------------------
Cumulative N-gram scores
Bleu-2:  0.2872074906291925
Bleu-3:  0.27505149679527074
Bleu-4:  0.2509128600915831


### Load our Saved model

In [5]:
model = T5Model("mt5", "/content/drive/MyDrive/MT5/Runs/glamorous-surf-26/best_model")

### Human Judgement

In [64]:
to_predict_test= [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(test_mt5["prefix"].tolist(), test_mt5["input_text"].tolist())
]
to_predict_test[62]
#type(to_predict[0])

'answer question: question: محرم نساء زواج  context: ولا تنكحوا ما نكح آباؤكم من النساء إلا ما قد سلف إنه كان فاحشة ومقتا وساء سبيلا. حرمت عليكم أمهاتكم وبناتكم وأخواتكم وعماتكم وخالاتكم وبنات الأخ وبنات الأخت وأمهاتكم اللاتي أرضعنكم وأخواتكم من الرضاعة وأمهات نسائكم وربائبكم اللاتي في حجوركم من نسائكم اللاتي دخلتم بهن فإن لم تكونوا دخلتم بهن فلا جناح عليكم وحلائل أبنائكم الذين من أصلابكم وأن تجمعوا بين الأختين إلا ما قد سلف إن الله كان غفورا رحيما. والمحصنات من النساء إلا ما ملكت أيمانكم كتاب الله عليكم وأحل لكم ما وراء ذلكم أن تبتغوا بأموالكم محصنين غير مسافحين فما استمتعتم به منهن فآتوهن أجورهن فريضة ولا جناح عليكم فيما تراضيتم به من بعد الفريضة إن الله كان عليما حكيما. '

In [79]:
import random
random.seed(44)
index_test_100 = random.sample(range(0, test_mt5.shape[0]), 100)
print(index_test_100)

[209, 266, 59, 90, 194, 115, 148, 14, 268, 271, 4, 51, 80, 155, 193, 207, 166, 183, 37, 86, 215, 72, 158, 40, 28, 130, 17, 150, 24, 69, 261, 27, 241, 160, 210, 74, 8, 88, 98, 245, 122, 121, 48, 231, 33, 66, 191, 63, 198, 12, 11, 172, 273, 174, 111, 181, 9, 189, 182, 177, 188, 167, 258, 84, 94, 120, 161, 116, 151, 138, 38, 264, 123, 206, 213, 175, 236, 157, 199, 223, 71, 250, 2, 57, 3, 267, 154, 165, 142, 139, 252, 91, 144, 50, 103, 249, 92, 87, 178, 190]


In [80]:
to_predict_test_100 = [to_predict_test[x] for x in index_test_100]
to_predict_test_100

['answer question: question: دليل قرآن تأليف سيد محمد  context: لقد أنزلنا آيات مبينات والله يهدي من يشاء إلى صراط مستقيم. ويقولون آمنا بالله وبالرسول وأطعنا ثم يتولى فريق منهم من بعد ذلك وما أولئك بالمؤمنين. وإذا دعوا إلى الله ورسوله ليحكم بينهم إذا فريق منهم معرضون. وإن يكن لهم الحق يأتوا إليه مذعنين. أفي قلوبهم مرض أم ارتابوا أم يخافون أن يحيف الله عليهم ورسوله بل أولئك هم الظالمون. إنما كان قول المؤمنين إذا دعوا إلى الله ورسوله ليحكم بينهم أن يقولوا سمعنا وأطعنا وأولئك هم المفلحون. ومن يطع الله ورسوله ويخش الله ويتقه فأولئك هم الفائزون. وأقسموا بالله جهد أيمانهم لئن أمرتهم ليخرجن قل لا تقسموا طاعة معروفة إن الله خبير بما تعملون. قل أطيعوا الله وأطيعوا الرسول فإن تولوا فإنما عليه ما حمل وعليكم ما حملتم وإن تطيعوه تهتدوا وما على الرسول إلا البلاغ المبين. ',
 'answer question: question: دليل مسيح صلب  context: يسألك أهل الكتاب أن تنزل عليهم كتابا من السماء فقد سألوا موسى أكبر من ذلك فقالوا أرنا الله جهرة فأخذتهم الصاعقة بظلمهم ثم اتخذوا العجل من بعد ما جاءتهم البينات فعفونا عن ذلك وآت

In [81]:
y_test = model.predict(to_predict_test_100)
print(y_test)

Generating outputs:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3557: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/100 [00:00<?, ?it/s]

['لقد أنزلنا آيات مبينات والله يهدي من يشاء إل', 'رفعنا فوقهم الطور بميثاقهم', 'لقد صدق عليهم إبليس ظنه فاتبعوه إلا فريق', 'شجرت الزقوم', 'قرآن الفجر', 'جاهدوا بأموالهم وأنفسهم في سبيل الله', 'إسماعيل', 'ذلك اليوم الحق فمن شاء اتخذ إلى ربه مآبا', 'لقد ذرأنا لجهنم كثيرا من الجن والإنس لهم قلوب', 'الذي عليه الحق سفيها أو ضعيفا أو لا يستطيع أن يمل', 'سليمان', 'يوسف', 'ثمود', 'يدخلون الجنة ولا يدخلون الجنة ولا يظلون', 'من كان في هذه أعمى فهو في الآخرة أعمى وأضل سب', 'طائفة من المؤمنين', 'إن الذين اتقوا إذا مسهم طائف من الشيطان تذكروا', 'لقد أرسلنا رسلا من قبلك وجعلنا لهم أزواجا', 'الذين استجابوا لله والرسول من بعد ما أصابهم القرح', 'يخرج الميت من الميت ويخرج الميت من الحي', 'الذي فرض عليك القرآن لرادك إلى معاد قل ربي', 'إخوة فلأمه السدس', 'لمن حولها', 'من عاقب بمثل ما عوقب به ثم بغي عليه لينصرنه الله', 'اتقوا الله وذروا ما بقي من الربا', 'الذين تابوا وأصلحوا وبينوا', 'إسماعيل', 'جعل منهم القردة والخنازير وعبد الطاغوت أولئك شر', 'ليس البر أن تولوا وجوهكم قبل المشرق والمغرب ولكن البر من آ', 

In [82]:
len(y_test)

100

In [83]:
context =  [test['passage'].tolist()[x] for x in index_test_100]
question =  [data_origin_test['question'].tolist()[x] for x in index_test_100]
dictionary = {'Context': context, 'Question': question, 'Answer': y_test}
hj = pd.DataFrame(dictionary)
hj.head()

,Context,Question,Answer
0,لقد أنزلنا آيات مبينات والله يهدي من يشاء إلى ...,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,لقد أنزلنا آيات مبينات والله يهدي من يشاء إل
1,يسألك أهل الكتاب أن تنزل عليهم كتابا من السماء...,ما الدلائل على أن المسيح لم يصلب؟,رفعنا فوقهم الطور بميثاقهم
2,ولقد صدق عليهم إبليس ظنه فاتبعوه إلا فريقا من ...,ما هي شروط الشفاعة؟,لقد صدق عليهم إبليس ظنه فاتبعوه إلا فريق
3,إن يوم الفصل ميقاتهم أجمعين. يوم لا يغني مولى ...,ما هي شجرة الزقوم؟,شجرت الزقوم
4,أقم الصلاة لدلوك الشمس إلى غسق الليل وقرآن الف...,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,قرآن الفجر


In [84]:
hj.to_csv('/content/drive/MyDrive/MT5/Human Judgement/Human_judgement.csv')